# rapyuta.instr
- [photometry_profile](#photometry_profile)
- [akari.cupid](#akari.cupid)

In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import sys
from pathlib import Path
import numpy as np

## Local
import rapyuta.utbox as UT
import rapyuta.plots as PL
from rapyuta.inout import fitsext, h5ext
from rapyuta.instr import (
    intercalib, photometry_profile,
)
from rapyuta.instr.akari import cupid

%matplotlib inline

## fold inputs
# UT.codefold(True, 'onclick')

                                                                                
                                   \  \|/  /                                    
                                \ _\_ ||| _/_ /                                 
                                      |||                                       
                                  /\/-/|\-\/\                                   
                              _ | _M_ _|_ _M_ | _                               
                            / .~.    _ ^ _    .~. \                             
                           |_|_0_|  |_&_&_|  |_0_|_|                            
                          //    .~.    o    .~.    \\                           
                         |_M_ _|_?_|__[_]__|_?_|_ _M_|                          
                                |             |                                 
                                 \  rapyuta  /                                  
                            

In [2]:
## Set dir
cfd = UT.getcfd('')
datdir = cfd+'/lib'
outdir = cfd+'/out/test_instr'
Path(outdir).mkdir(parents=True, exist_ok=True)

## photometry_profile

## AKARI
### akari.cupid

In [ ]:
parobs = [
    ['3390001.1','NG','F011100297_N002','Ns',24,2,5],
    ['3390001.1','NG','F011100297_N002','Nh',28,7,3],
]
buildir = outdir+'/cubuild/'
if not os.path.exists(buildir):
    os.makedirs(buildir)
fits_irc = []
for obs in parobs:
    fits_irc.append(obs[0]+'_'+obs[3])
## MC add pointing unc
Nmc = 2
for i in range(len(parobs)):
    for j in range(Nmc+1):
        if j==0:
            verbose = False
        else:
            verbose = False
        
        cup = cupid(datdir, obsid=parobs[i][0], slit=parobs[i][3],
                    spec=parobs[i][1], imref=parobs[i][2], verbose=verbose)
        if j==0:
            cup.spec_build(buildir+fits_irc[i],
                           filRAW=buildir+'raw', tmpdir=buildir, fiLOG=buildir+'build_history',
                           Nx=parobs[i][6], Ny=parobs[i][4], Nsub=parobs[i][5],
                           pixscale=1, wmax=4.3, supix=True)
        else:
            cup.spec_build(buildir+fits_irc[i]+'_'+str(j), tmpdir=buildir,
                           Nx=parobs[i][6], Ny=parobs[i][4], Nsub=parobs[i][5],
                           pixscale=1, wmax=4.3, supix=True,
                           dist='splitnorm', sig_pt=3, fill_pt='med', swarp=False)
    mcimage = []
    for j in range(Nmc+1):
        if j==0:
            hdr = read_fits(buildir+fits_irc[i]).header
            wvl = read_fits(buildir+fits_irc[i]).wave
            data = read_fits(buildir+fits_irc[i]).data
            write_fits(outdir+fits_irc[i], hdr, data, wvl)
        else:
            mcimage.append(read_fits(buildir+fits_irc[i]+'_'+str(j)).data)
    if Nmc>1:
        mcimage = np.array(mcimage)
        unc = np.nanstd(mcimage, axis=0)
        write_fits(outdir+fits_irc[i]+'_unc', hdr, unc, wvl)